In [1]:
import numpy as np
import matplotlib.pyplot as plt

class Markov:
    def __init__(self):
        self.iNrStates = None
        self.iMaxTime  = None    
        self.dPij = []
        self.dPre = []
        self.dPost= []
        self.dv   = []
        self.dDK  = []
        self.dDKDistr  = []
        
        self.bCalculated = False
        self.bCalculatedDistr = False
        self.iStart = None
        self.iStop  = None
        self.fDistrLow = -1000
        self.fDistrHigh = 30000
        self.iNrBuckets = 3300
        self.fBucketWidth = (self.fDistrHigh-self.fDistrLow)/self.iNrBuckets
        self.fBucketWidthRound = self.fBucketWidth / 2.
                
    def vDefineModel(self,iNrStates,iMaxTime=1200):
        self.iNrStates = iNrStates
        self.iMaxTime = iMaxTime
        for i in range(iMaxTime):
            tempPij = np.zeros([iNrStates,iNrStates])
            tempPost = np.zeros([iNrStates,iNrStates])
            tempPre = np.zeros([iNrStates])
            tempDK = np.zeros([iNrStates])
            self.dPij.append(tempPij)
            self.dPost.append(tempPost)
            self.dPre.append(tempPre)
            self.dDK.append(tempDK)
        tempv = np.zeros([iMaxTime])
        self.dv=tempv
        
    def iBucketNr(self, fValue):
        if fValue < self.fDistrLow:
            return(0)
        iBNR = (int(min(self.iNrBuckets-1,(fValue-self.fDistrLow)/self.fBucketWidth+self.fBucketWidthRound)))
        return(iBNR)
    
    def fValueOfBucket(self, iBucket):
        return(self.fBucketWidth*min(self.iNrBuckets-1,iBucket)+self.fDistrLow)
    
    def vCreateDistModel(self):
        print("You Know that you can call me only once everything is done")
        for i in range(self.iMaxTime):
            tempDK = np.zeros([self.iNrStates,self.iNrBuckets])
            self.dDKDistr.append(tempDK)
    
    def vSetDiscount(self,fIRate):
        vTemp = 1./(1.+fIRate)
        print("Discount %.4f"%(vTemp))
        for i in range(self.iMaxTime):
            self.dv[i] = vTemp
        self.bCalculated = False
    
    def vSetPij(self,t,i,j,fValue):
        self.dPij[t][i,j] = fValue
        self.bCalculated = False
    
    def vSetPre(self,t,i,j,fValue):
        self.dPre[t][i] = fValue
        self.bCalculated = False
    
    def vSetPost(self,t,i,j,fValue):
        self.dPost[t][i,j] = fValue
        self.bCalculated = False
    
    
    def doCalculateDK(self,iStart,iStop,iAge,iState):
        self.iStop = iStop
        self.iStart = iStart
        self.bCalculated = True
        for i in range(self.iMaxTime):
            self.dDK[i] *= 0.
        
        for i in range(self.iStart-1, self.iStop-1,-1):
            #print("Calc Time", i)
            for j in range(self.iNrStates):
                self.dDK[i][j] = self.dPre[i][j]
                for k in range(self.iNrStates):
                    self.dDK[i][j] += self.dv[i]*self.dPij[i][j,k]*(self.dPost[i][j,k]+self.dDK[i+1][k])
                    
    def doCalculateDKDistr(self,iStart,iStop,iAge,iState):
        self.iStop = iStop
        self.iStart = iStart
        self.bCalculatedDistr = True
        self.vCreateDistModel()
        for i in range(self.iMaxTime):
            self.dDKDistr[i] *= 0.
        # Set Boundary Conditions
        iIndexSwitch = self.iBucketNr(0)
        for j in range(self.iNrStates):
            value = 0.
            for l in range(self.iNrBuckets):
                if l > iIndexSwitch:
                           value = 1.
                self.dDKDistr[self.iStart][j,l] = value
        # Calculation                   
        for i in range(self.iStart-1, self.iStop-1,-1):
            print("Dirst DK Calc Time", i)
            for j in range(self.iNrStates):
                for k in range(self.iNrStates):
                    for l in range(self.iNrBuckets):
                        dNewXTPlusOne = (self.fValueOfBucket(l) - self.dPre[i][j])/self.dv[i] - self.dPost[i][j,k]
                        self.dDKDistr[i][j,l] += self.dPij[i][j,k]*(self.dDKDistr[i+1][k,self.iBucketNr(dNewXTPlusOne)])
                                      
    
    def dGetDK(self,iStart,iStop,iAge,iState):
        if (iStart != self.iStart or iStop != self.iStop or not(self.bCalculated)):
            self.doCalculateDK(iStart,iStop,iAge,iState)
        return(self.dDK[iAge][iState])
    
    def dGetDKDistr(self,iStart,iStop,iAge,iState,fValue):
        if (iStart != self.iStart or iStop != self.iStop or not(self.bCalculatedDistr)):
            temp = self.dGetDK(iStart,iStop,iAge,iState) # To be on the safe side
            self.doCalculateDKDistr(iStart,iStop,iAge,iState)
        return(self.dDKDistr[iAge][iState,self.iBucketNr(fValue)])
    
    def PrintDKs(self,iStart,iStop):
        for i in range(iStop,iStart+1):
            strTemp = " %3d :"%(i)
            for j in range(self.iNrStates):
                 strTemp += "  %7.0f "%(self.dGetDK(iStart,iStop,i,j))
            print(strTemp)
    
    def PlotDKs(self,iStart,iStop,figNr=1):
        x = []
        y = []
        for i in range(iStop,iStart+1):
            x.append(i)
            ytemp = np.zeros(self.iNrStates)
            for j in range(self.iNrStates):
                ytemp[j] = self.dGetDK(iStart,iStop,i,j)
            y.append(ytemp)
        plt.figure(figNr)
        plt.plot(x,y)
        plt.grid(True)
    
    def PlotDKDistr(self,iStart,iStop, iSteps = None, iStates = [0], iDeltaT = 5, figNr=10, eps = 0.01,legTitle=""):
        if iSteps == None:
            iSteps = []
            for i in range(iStop,iStart,iDeltaT):
                iSteps.append(i)
            iSteps.append(iStart)
        for i in iSteps:
            for j in iStates:
                x = []
                y = []
                for k in range(self.iNrBuckets):
                    xLoc = eps + self.fValueOfBucket(k)
                    yLoc = self.dGetDKDistr(iStart,iStop,i,j,xLoc)
                    x.append(xLoc)
                    y.append(yLoc)
    
                plt.figure(figNr)
                plt.plot(x,y)
                plt.grid(True)
                mylegend = legTitle + "Age %d - State %d"%(i,j)
                plt.title(mylegend)
                figNr+=1
        
def mu(x):
    return(np.exp(-7.85785 + 0.01538*x + 0.000577355*x**2))

symMB = Markov()
symMB.vDefineModel(8)
symMB.vSetDiscount(0.04)


iStop = 30
AnnuityOrphan = 800
AnnuityOrphanMother = 600
AnnuityOrphanBoth = 1200
iDxy = -3 
iDxz = -29 # child at beginning has an age of 30-29 =1 
iSz  = 25
iS   = (iSz - (iStop+iDxz))+iStop
iStart = iS

def iState(bFatherAlive,bMotherAlive,bChildAlive):
    iState = 0
    if not(bFatherAlive): iState +=1
    if not(bMotherAlive): iState +=2
    if not(bChildAlive): iState +=4
    return(iState)    

for i in range(iStop,iStart):
    qx = mu(i)
    qy = mu(i+iDxy)
    qz = mu(i+iDxz)
    symMB.vSetPij(i,iState(True,True,True),iState(True,True,True),(1-qx)*(1-qy)*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(False,True,True),qx*(1-qy)*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(True,False,True),(1-qx)*qy*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(True,True,False),(1-qx)*(1-qy)*qz)
    symMB.vSetPij(i,iState(True,True,True),iState(False,False,True),qx*qy*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(True,False,False),(1-qx)*qy*qz)
    symMB.vSetPij(i,iState(True,True,True),iState(False,True,False),qx*(1-qy)*qz)
    symMB.vSetPij(i,iState(True,True,True),iState(False,False,False),qx*qy*qz)
    
    symMB.vSetPij(i,iState(False,True,True),iState(False,True,True),(1-qy)*(1-qz))
    symMB.vSetPij(i,iState(False,True,True),iState(False,False,True),(qy)*(1-qz))
    symMB.vSetPij(i,iState(False,True,True),iState(False,True,False),(1-qy)*(qz))
    symMB.vSetPij(i,iState(False,True,True),iState(False,False,False),(qy)*(qz))
                  
    symMB.vSetPij(i,iState(True,False,True),iState(True,False,True),(1-qx)*(1-qz))
    symMB.vSetPij(i,iState(True,False,True),iState(False,False,True),qx*(1-qz))
    symMB.vSetPij(i,iState(True,False,True),iState(True,False,False),(1-qx)*qz)
    symMB.vSetPij(i,iState(True,False,True),iState(False,False,False),(qx)*(qz))
                  
    symMB.vSetPij(i,iState(True,True,False),iState(True,True,False),(1-qx)*(1-qy))
    symMB.vSetPij(i,iState(True,True,False),iState(False,True,False),(qx)*(1-qy))
    symMB.vSetPij(i,iState(True,True,False),iState(True,False,False),(1-qx)*(qy))
    symMB.vSetPij(i,iState(True,True,False),iState(False,False,False),(qx)*(qy))
    
    symMB.vSetPij(i,iState(True,False,False),iState(True,False,False),(1-qx))
    symMB.vSetPij(i,iState(True,False,False),iState(False,False,False),(qx))
    
    symMB.vSetPij(i,iState(False,True,False),iState(False,True,False),(1-qy))
    symMB.vSetPij(i,iState(False,True,False),iState(False,False,False),(qy))
                  
    symMB.vSetPij(i,iState(False,False,True),iState(False,False,True),(1-qz))
    symMB.vSetPij(i,iState(False,False,True),iState(False,False,False),(qz))
     
    symMB.vSetPij(i,iState(False,False,False),iState(False,False,False), 1.)

    
for i in range(iStop,iStart):
    symMB.vSetPre(i,iState(False,True,True),0,AnnuityOrphan)
    symMB.vSetPre(i,iState(True,False,True),1,AnnuityOrphanMother)
    symMB.vSetPre(i,iState(False,False,True),2,AnnuityOrphanBoth)

symMB.PrintDKs(iStart,iStop)

#symMB.PlotDKDistr(iStart,iStop,iSteps = [30], iStates = [0,1,2,3])

Discount 0.9615
  30 :      310     12709      9615     18944         0         0         0         0 
  31 :      302     12385      9370     18461         0         0         0         0 
  32 :      293     12048      9115     17958         0         0         0         0 
  33 :      284     11697      8849     17436         0         0         0         0 
  34 :      273     11332      8573     16892         0         0         0         0 
  35 :      262     10952      8285     16327         0         0         0         0 
  36 :      250     10557      7985     15738         0         0         0         0 
  37 :      237     10145      7673     15127         0         0         0         0 
  38 :      223      9717      7349     14490         0         0         0         0 
  39 :      209      9272      7011     13828         0         0         0         0 
  40 :      194      8809      6660     13140         0         0         0         0 
  41 :      177      8328  

In [2]:
for i in range(iStop,iStart):
    qx = mu(i)
    qy = mu(i+iDxy)
    qz = mu(i+iDxz)
    # want qy higher if man death and vice versa
    alpha = 1.15
    # also qz higher if orphan
    beta = 1.05
    symMB.vSetPij(i,iState(True,True,True),iState(True,True,True),(1-qx)*(1-qy)*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(False,True,True),qx*(1-qy)*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(True,False,True),(1-qx)*qy*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(True,True,False),(1-qx)*(1-qy)*qz)
    symMB.vSetPij(i,iState(True,True,True),iState(False,False,True),qx*qy*(1-qz))
    symMB.vSetPij(i,iState(True,True,True),iState(True,False,False),(1-qx)*qy*qz)
    symMB.vSetPij(i,iState(True,True,True),iState(False,True,False),qx*(1-qy)*qz)
    symMB.vSetPij(i,iState(True,True,True),iState(False,False,False),qx*qy*qz)
    
    symMB.vSetPij(i,iState(False,True,True),iState(False,True,True),(1-alpha * qy)*(1-beta*qz))
    symMB.vSetPij(i,iState(False,True,True),iState(False,False,True),(alpha * qy)*(1-beta*qz))
    symMB.vSetPij(i,iState(False,True,True),iState(False,True,False),(1-alpha * qy)*(beta*qz))
    symMB.vSetPij(i,iState(False,True,True),iState(False,False,False),(alpha * qy)*(beta*qz))
                  
    symMB.vSetPij(i,iState(True,False,True),iState(True,False,True),(1-alpha * qx)*(1-beta*qz))
    symMB.vSetPij(i,iState(True,False,True),iState(False,False,True),alpha * qx*(1-beta*qz))
    symMB.vSetPij(i,iState(True,False,True),iState(True,False,False),(1-alpha * qx)*beta*qz)
    symMB.vSetPij(i,iState(True,False,True),iState(False,False,False),(alpha * qx)*(beta*qz))
    # lazy not needed         
    symMB.vSetPij(i,iState(True,True,False),iState(True,True,False),(1-qx)*(1-qy))
    symMB.vSetPij(i,iState(True,True,False),iState(False,True,False),(qx)*(1-qy))
    symMB.vSetPij(i,iState(True,True,False),iState(True,False,False),(1-qx)*(qy))
    symMB.vSetPij(i,iState(True,True,False),iState(False,False,False),(qx)*(qy))
    
    symMB.vSetPij(i,iState(True,False,False),iState(True,False,False),(1-qx))
    symMB.vSetPij(i,iState(True,False,False),iState(False,False,False),(qx))
    
    symMB.vSetPij(i,iState(False,True,False),iState(False,True,False),(1-qy))
    symMB.vSetPij(i,iState(False,True,False),iState(False,False,False),(qy))
    #still beta or even higher?              
    symMB.vSetPij(i,iState(False,False,True),iState(False,False,True),(1-beta*qz))
    symMB.vSetPij(i,iState(False,False,True),iState(False,False,False),(beta*qz))
     
    symMB.vSetPij(i,iState(False,False,False),iState(False,False,False), 1.)

    
for i in range(iStop,iStart):
    symMB.vSetPre(i,iState(False,True,True),0,AnnuityOrphan)
    symMB.vSetPre(i,iState(True,False,True),1,AnnuityOrphanMother)
    symMB.vSetPre(i,iState(False,False,True),2,AnnuityOrphanBoth)

symMB.PrintDKs(iStart,iStop)


  30 :      311     12718      9634     18939         0         0         0         0 
  31 :      302     12394      9388     18456         0         0         0         0 
  32 :      294     12056      9133     17954         0         0         0         0 
  33 :      284     11705      8867     17432         0         0         0         0 
  34 :      274     11340      8589     16889         0         0         0         0 
  35 :      262     10960      8301     16323         0         0         0         0 
  36 :      250     10564      8001     15735         0         0         0         0 
  37 :      237     10152      7688     15124         0         0         0         0 
  38 :      224      9724      7363     14488         0         0         0         0 
  39 :      209      9279      7024     13826         0         0         0         0 
  40 :      194      8815      6672     13138         0         0         0         0 
  41 :      178      8333      6306     124